In [1]:
# -*- coding: utf-8 -*-
import argparse
import os
import string
import sys
os.chdir('/home/hyeryung_son/mucoco')
from itertools import repeat
import torch.multiprocessing as mp
from typing import List
from itertools import permutations,product

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM, AutoTokenizer, AutoConfig

from new_module.utils.robertacustom import RobertaCustomForSequenceClassification
from new_module.locate.new_locate_utils import *
import new_module.losses as lossbuilder

import logging
import os
from typing import List
from tqdm import tqdm

import torch
import torch.nn.functional as F
import transformers

In [2]:
## setup for prototyping
logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOGGING_LEVEL", logging.DEBUG))

config={#'model_paths':['gpt2-large','/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint'],
        # 'tokenizer_paths':['gpt2-large','/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint'],
        'model_paths':['gpt2-large','models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best'],
        'tokenizer_paths':['gpt2-large','models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best'],
        'model_types': ["AutoModelForCausalLM", "AutoModelForSequenceClassification"],
        'cache_dir': "hf_cache",
        'target_type': "embeds",
        'method': "mlm-beamsearch-v0",
       'losses': ["gpt2", "classification_no_prefix_logprobloss"],
       'target_label_ids': [0,0] ,
       'build_loss_dict': {"coeff_steps": 200, "coeff_pattern": "constant", "loss_type": "xentropy", "length_normalize": False, "AR_temperature": 1.0, "AR_top_k": 0, "AR_top_p": 0.96, "max_output_length": 20},
       'min_epsilons': [0.75],
       'source_data': 'new_module/data/toxicity-avoidance/testset_gpt2_2500.jsonl',
       'locate_unit': 'token',
       'locate_method': 'grad_norm',
       'device': 'cuda',
       'k_per_location': 3,
       'closs_weight': 0.9,
       'beam_size': 3}

class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

In [3]:
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []
primary_model = None
for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[model_path] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[model_path] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if "Custom" in config["model_types"][i]:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(utils, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].cuda()

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

    if i == 0:
        primary_model = name2model[model_path]

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").to(config['device'])  

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["model_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer

primary_tokenizer = name2tokenizer['gpt2-large']
secondary_tokenizer = list(name2tokenizer.values())[1]

Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best were not used when initializing RobertaForSequenceClassification: ['roberta.embeddings.word_embeddings.0.weight', 'roberta.embeddings.word_embeddings.1.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to

In [4]:
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    outputs_df['generations']=outputs_df['generations'].apply(lambda x: x['text'] if isinstance(x, dict) else x)
    outputs_df = outputs_df.dropna().reset_index(drop=True)
    return outputs_df
toxic_gen=pd.read_json('new_module/data/toxicity-avoidance/dev_set.jsonl',lines=True)
toxic_gen=unravel(toxic_gen)
source_text = '“I went to use the bathroom, and I went in and Lance was bent'
prediction = toxic_gen.loc[toxic_gen['prompt']==source_text,'generations'].tolist()
ckpt_path = config['model_paths'][1]
model = name2model[ckpt_path] #AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = name2tokenizer[ckpt_path] #AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)
loc_machine=LocateMachine(model,tokenizer)
masked_text = loc_machine.locate_main(prediction, "grad_norm", max_num_tokens = 6, unit=config['locate_unit'], num_layer=10, label_id=0)

In [5]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 
masked_sequence=inputs['input_ids']

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=config['k_per_location'],
    dim=-1,
)
# print(f"predicted_token_ids: {predicted_token_ids}")
# print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


In [36]:
def beam_rerank(source_text:str, 
                    masked_sequence:torch.Tensor, 
                    indices_in_mlm_tokens:tuple,
                    # predicted_token_ids:torch.return_types.topk,
                    predicted_token_ids:torch.Tensor,
                    mlm_tokenizer:transformers.AutoTokenizer, 
                    lossfns:List[lossbuilder.BaseLoss],
                    config:dict):
    """params: 
    source_text(prompt) should be text. 
    masked_sequence should be token ids tokenized by MLM's tokenizer.
    indices_in_mlm_tokens should be a result of running 
    `indices_in_mlm_tokens = (
                                inputs.input_ids == mlm_tokenizer.mask_token_id
                                ).nonzero(as_tuple=True)`
    predicted_token_ids should be a result of running
    `predicted_token_ids = torch.topk(
                            logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
                            k=config['k_per_location'],
                            dim=-1,).indices`
    """
    
    hypotheses = masked_sequence[:, None, :].repeat((1,config['beam_size'],1))
    edit_indices = sorted(list(set(indices_in_mlm_tokens[1].tolist())))

    for i in tqdm(edit_indices,total=len(edit_indices)):
        
        batch_ids_to_edit = indices_in_mlm_tokens[0][indices_in_mlm_tokens[1]==i]

        tmp_hypotheses = hypotheses[batch_ids_to_edit].detach().clone()
        tmp_hypotheses=tmp_hypotheses.repeat((1,config['k_per_location'],1))

        # candidates = predicted_token_ids.indices[(indices_in_mlm_tokens[1]==i).nonzero().squeeze(-1),:]
        candidates = predicted_token_ids[(indices_in_mlm_tokens[1]==i).nonzero().squeeze(-1),:]
        candidates = candidates[:, :, None].repeat((1,1, config['beam_size'])).reshape(candidates.shape[0], -1,1)

        tmp_hypotheses = torch.cat((tmp_hypotheses[:, :, :i], candidates),dim=-1) ## tmp_hypotheses: [(a,b,c),(a,b,c), ..., (a,b,c)], candidates: [(p,p,p), (q,q,q), ..., (v,v,v)]
        tmp_hypotheses = tmp_hypotheses.reshape(-1, tmp_hypotheses.shape[-1])
        
        loss_weights = [1 - config['closs_weight'], config['closs_weight']]
        curr_loss = torch.zeros(tmp_hypotheses.shape[0]).to(config['device'])
        for lossid, lossname in enumerate(config["losses"]):
            if config['method'] == 'mlm-beamsearch-v1' and lossid > 0:
                continue
            with torch.no_grad():
                lossvalue = lossfns[lossid].compute_gold_loss(
                    source_text, mlm_tokenizer.batch_decode(tmp_hypotheses,skip_special_tokens=True),
                    label_id=config['target_label_ids'][lossid],
                )
            torch.cuda.empty_cache()
            curr_loss += loss_weights[lossid] * lossvalue
            
        curr_loss = torch.stack(torch.split(curr_loss, config['beam_size'] * config['k_per_location'], dim=0),dim=0)
        top_beams=torch.topk(curr_loss, k=(config['beam_size']*2+1), dim=-1, largest=False).indices

        tmp_hypotheses = torch.split(tmp_hypotheses, config['beam_size'] * config['k_per_location'], dim=0) ## 아래의 작업을 더 간단히 할 수 있는 방법?
        tmp_hypotheses = torch.stack([x[top_beams[j]] for j, x in enumerate(tmp_hypotheses)],dim=0)
        tmp_hypotheses = torch.unique(tmp_hypotheses, dim=1)[:, :config['beam_size'], :]
        
        hypotheses[batch_ids_to_edit,:, i]=tmp_hypotheses[:, :, i]
            
    return mlm_tokenizer.batch_decode(hypotheses.reshape(-1, hypotheses.shape[-1]), skip_special_tokens=True) ## return 값에 대한 의문..!

In [38]:
hypotheses = beam_rerank(source_text, 
                        masked_sequence, 
                        indices_in_mlm_tokens,
                        predicted_token_ids.indices,
                        mlm_tokenizer, 
                        lossfns,
                        config)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 32, 33]


  0%|          | 0/30 [00:00<?, ?it/s]

1
['Behind', 'Behind', 'Behind', 'Let', 'Let', 'Let', 'With', 'With', 'With', 'Give', 'Give', 'Give', 'Put', 'Put', 'Put', 'Watch', 'Watch', 'Watch', 'Leave', 'Leave', 'Leave', 'Tell', 'Tell', 'Tell', 'For', 'For', 'For', 'To', 'To', 'To', 'I', 'I', 'I', 'He', 'He', 'He', '"', '"', '"', 'She', 'She', 'She', 'We', 'We', 'We', 'They', 'They', 'They', ' "', ' "', ' "', 'Jason', 'Jason', 'Jason', 'Mike', 'Mike', 'Mike', 'Michael', 'Michael', 'Michael', 'And', 'And', 'And', 'Clean', 'Clean', 'Clean', 'Even', 'Even', 'Even', 'But', 'But', 'But', 'In', 'In', 'In', 'See', 'See', 'See', 'Except', 'Except', 'Except', 'Take', 'Take', 'Take', 'Check', 'Check', 'Check', 'Under', 'Under', 'Under', '"', '"', '"', 'He', 'He', 'He', 'I', 'I', 'I', '[', '[', '[', 'H', 'H', 'H', 'The', 'The', 'The', '(', '(', '(', 'And', 'And', 'And', 'Oh', 'Oh', 'Oh', 'D', 'D', 'D', 'The', 'The', 'The', 'A', 'A', 'A', 'This', 'This', 'This', 'D', 'D', 'D', 'C', 'C', 'C', 'R', 'R', 'R', 'B', 'B', 'B', 'H', 'H', 'H', 'P',

  3%|▎         | 1/30 [00:00<00:16,  1.74it/s]

['For', 'Watch', 'Behind', 'I', '"', ' "', 'But', 'In', 'And', '(', '"', '[', 'C', 'H', 'P']
2
['I was', '" was', ' " was', "I's", '"\'s', ' "\'s', "I'm", '"\'m', ' "\'m', 'IHe', '"He', ' "He', 'IStart', '"Start', ' "Start', 'I starts', '" starts', ' " starts', 'I started', '" started', ' " started', 'IStop', '"Stop', ' "Stop', 'II', '"I', ' "I', 'I is', '" is', ' " is', '(,', '",', '[,', "('s", '"\'s', "['s", '(.', '".', '[.', '(:', '":', '[:', '( was', '" was', '[ was', '( is', '" is', '[ is', '( the', '" the', '[ the', '(G', '"G', '[G', '(-', '"-', '[-', '( he', '" he', '[ he', 'C:', 'H:', 'P:', 'C the', 'H the', 'P the', 'C of', 'H of', 'P of', 'C-', 'H-', 'P-', "C's", "H's", "P's", 'CThe', 'HThe', 'PThe', 'C,', 'H,', 'P,', 'C R', 'H R', 'P R', 'C a', 'H a', 'P a', 'C is', 'H is', 'P is']


...

[' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what that', ' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what it', ' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what he']
32
[' over me and put his hand in my butt and he was running his hand up and down my ass and he just put his hand back in my butt and he', ' over me and put his hand in my butt and he was moving his hand up and down my ass and I then put his hand back in my butt and he', ' over me and put his hand in my butt and he was moving his hand up and down my ass and he also put his hand back in my butt and he', ' over me and put his hand in my butt and he was running his hand up and down my ass and he just put his hand back in my butt and just', ' over me and put his hand in my butt and he was moving his hand up and down my ass and I then put his 

 97%|█████████▋| 29/30 [00:12<00:00,  2.91it/s]

[' over me and put his hand in my butt and he was running his hand up and down my ass and he just put his hand back in my butt and he', ' over me and put his hand in my butt and he was running his hand up and down my ass and he just put his hand back in my butt and I', ' over me and put his hand in my butt and he was running his hand up and down my ass and he just put his hand back in my butt and just']
33
[' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what that was going.', ' over me and he was choking me. And I pulled away and he was fighting me, and I just in up I he had no idea what it was going.', ' over me and he was choking me. And I pulled away and he was fighting me, and I got just and like, had no idea what he was going.', ' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what that was going on', ' over me and he was choking me. And I pulled away and he was fi

100%|██████████| 30/30 [00:13<00:00,  2.25it/s]

[' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what that was going to', ' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what that was going on', ' over me and he was pushing me. And I pulled away and he was fighting me, and I was to, and I had no idea what that was going through']


In [67]:
## combi_rerank 
def combi_rerank(masked_sequence:torch.Tensor, 
                 indices_in_mlm_tokens:tuple,
                 predicted_token_ids:torch.Tensor,
                 mlm_tokenizer:transformers.AutoTokenizer,
                 config:dict) -> List[str]:
    """params: 
    masked_sequence should be token ids tokenized by MLM's tokenizer.
    indices_in_mlm_tokens should be a result of running 
    `indices_in_mlm_tokens = (
                                inputs.input_ids == mlm_tokenizer.mask_token_id
                                ).nonzero(as_tuple=True)`
    predicted_token_ids should be a result of running
    `predicted_token_ids = torch.topk(
                            logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
                            k=config['k_per_location'],
                            dim=-1,).indices`
    """

    k = config['k_per_location']
    hypotheses = []
    num_batches = masked_sequence.shape[0]
    for i in range(num_batches):
        
        l = (indices_in_mlm_tokens[0] == i).sum().item()
        tok_cand_combos = list(product(range(k),repeat=l))
        
        tmp_hypotheses = masked_sequence[i,:].repeat((k**l,1))
        tmp_hypotheses[:, indices_in_mlm_tokens[1][indices_in_mlm_tokens[0] == i]] = \
            predicted_token_ids[indices_in_mlm_tokens[0] == i, tok_cand_combos]
            
        tmp_dec_seq = mlm_tokenizer.batch_decode(
                    tmp_hypotheses, skip_special_tokens=True
            )
        hypotheses.extend(tmp_dec_seq)
    return hypotheses

In [68]:
my_hypotheses = combi_rerank(masked_sequence, 
                            indices_in_mlm_tokens,
                            predicted_token_ids.indices,
                            mlm_tokenizer,
                            config)

In [69]:
my_hypotheses

['Just me trying to cum. I could hear him starting to come but I couldn stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come but I didn stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come but I can stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come and I couldn stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come and I didn stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come and I can stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come so I couldn stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to come so I didn stop him. Thank God he had these pants on me.',
 'Just me trying to cum. I could hear him starting to 

In [8]:
k = 3 # config['k_per_location']
loss_weights = [1 - config['closs_weight'], config['closs_weight']]
selection_criteria = "weighted_sum"
hypotheses = []
best_ixes = []
best_weighted_loss = []
best_allsat = []
best_logging_loss = []
num_batches = masked_sequence.shape[0]
for i in range(num_batches):
    print(i)
    l = (indices_in_mlm_tokens[0] == i).sum().item()
    tok_cand_combos = list(product(range(k),repeat=l))
    
    tmp_hypotheses = masked_sequence[i,:].repeat((k**l,1))
    tmp_hypotheses[:, indices_in_mlm_tokens[1][indices_in_mlm_tokens[0] == i]] = \
        predicted_token_ids.indices[indices_in_mlm_tokens[0] == i, tok_cand_combos]
        
    tmp_dec_seq = mlm_tokenizer.batch_decode(
                tmp_hypotheses, skip_special_tokens=True
        )
    
    del tmp_hypotheses
        
    curr_loss = torch.zeros(len(tmp_dec_seq)).to(config['device'])
    allsat = torch.zeros(len(tmp_dec_seq)).to(config['device'])
    logging_loss = torch.zeros((len(tmp_dec_seq),2)).to(config['device'])

    for lossid, lossname in enumerate(config["losses"]):
        with torch.no_grad():
            lossvalue = lossfns[lossid].compute_gold_loss(
                source_text, tmp_dec_seq,
                label_id=config['target_label_ids'][lossid],
            )
        torch.cuda.empty_cache()
        curr_loss += loss_weights[lossid] * lossvalue
        logging_loss[:, lossid] = lossvalue.clone()
        
        # if lossid==0:
        #     candidate_primary_losses = lossvalue.clone()
        # elif (lossid >= 1):
        if (lossid >= 1):
            allsat[torch.where(lossvalue > -np.log(config["min_epsilons"][lossid - 1]))[0]] = 1
            
    allsat_ix = allsat.nonzero().squeeze(0)
    if (allsat_ix.shape[0] > 0) and (selection_criteria == "allsat_primary"):
        best_ix = allsat_ix[curr_loss[allsat == 1].argmin()]
    else: ## in case selection_criteria == "weighted_sum" or allsat is all False
        best_ix = torch.argmin(curr_loss)

    
    hypotheses.append(tmp_dec_seq[best_ix])
    best_weighted_loss.append(curr_loss[best_ix])
    best_allsat.append(allsat[best_ix])
    best_logging_loss.append(logging_loss[best_ix])
    
    del curr_loss, allsat, logging_loss, tmp_dec_seq, tmp_hypotheses
    torch.cuda.empty_cache()

0


OutOfMemoryError: CUDA out of memory. Tried to allocate 670.00 MiB (GPU 0; 15.78 GiB total capacity; 14.11 GiB already allocated; 176.94 MiB free; 14.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF